问题:你想创建一个内嵌变量的字符串，变量被它的值所表示的字符串替换掉

In [1]:
s = '{name} has {n} messages.'
s.format(name='Guido', n=37)


'Guido has 37 messages.'

In [2]:
# 如果要被替换的变量能在变量域中找到， 那么你可以结合使用 format_map() 和 vars() 
# format_map() 参数为字典
name = 'Guido'
n = 40
s.format_map(vars())


'Guido has 40 messages.'

In [4]:
# vars() 还有一个有意思的特性就是它也适用于对象实例
class Info:
    def __init__(self, name, n):
        self.name = name
        self.n = n


a = Info('Guido', 33)
s.format_map(vars(a))


'Guido has 33 messages.'

In [5]:
# format 和 format_map() 的一个缺陷就是它们并不能很好的处理变量缺失的情况
# 即当缺失某个变量时，会报错

In [7]:
# 一种避免这种错误的方法是另外定义一个含有 __missing__() 方法的字典对象，就像下面这样


class safesub(dict):
    def __missing__(self, key):
        return '{' + key + '}'


{}
<class '__main__.safesub'>


In [14]:
# s.format_map(vars())
s.format_map(safesub(vars()))

'Guido has {n} messages.'

In [17]:
# 如果你发现自己在代码中频繁的执行这些步骤，你可以将变量替换步骤用一个工具函数封装起来。就像下面这样
# sys.getframe(1).f_locals获取的是函数的变量的名称
# sub() 函数使用 sys._getframe(1) 返回调用者的栈帧。可以从中访问属性 f_locals 来获得局部变量。 
# 毫无疑问绝大部分情况下在代码中去直接操作栈帧应该是不推荐的。 但是，对于像字符串替换工具函数而言它是非常有用的。 
# 另外，值得注意的是 f_locals 是一个复制调用函数的本地变量的字典。 
# 尽管你可以改变 f_locals 的内容，但是这个修改对于后面的变量访问没有任何影响。 
# 所以，虽说访问一个栈帧看上去很邪恶，但是对它的任何操作不会覆盖和改变调用者本地变量的值
import sys

def sub(text):
    return text.format_map(safesub(sys._getframe(1).f_locals))

name = 'Toxic'
n = 40
print(sub('Hello {name}'))
print(sub('You have {n} messages.'))
print(sub('Your favorite color is {color}'))


Hello Toxic
You have 40 messages.
Your favorite color is {color}


In [20]:
import string
s = string.Template('$name has $n messages.')
s.substitute(vars())

'Toxic has 39 messages.'